# Модуль 5.5 — Ретраи и backoff

**Цель:** сделать вызовы устойчивыми к временным сбоям.

**Что сделаем:**
- реализуем простой retry
- добавим экспоненциальную задержку
- покажем, как оборачивать LLM‑вызов

## 1) Простой retry

Попробуем повторить вызов, если он временно упал.

In [ ]:
import random
import time


def flaky_call():
    if random.random() < 0.6:
        raise RuntimeError("Временный сбой")
    return "Успех"


def retry(func, attempts=3, delay=1.0):
    for i in range(attempts):
        try:
            return func()
        except Exception as e:
            print(f"Попытка {i+1} не удалась:", e)
            if i == attempts - 1:
                raise
            time.sleep(delay)

print(retry(flaky_call, attempts=5, delay=0.5))

## 2) Экспоненциальный backoff

Чем больше попыток — тем дольше па

In [ ]:
def retry_backoff(func, attempts=4, base_delay=0.5):
    delay = base_delay
    for i in range(attempts):
        try:
            return func()
        except Exception as e:
            print(f"Попытка {i+1} не удалась:", e)
            if i == attempts - 1:
                raise
            time.sleep(delay)
            delay *= 2

print(retry_backoff(flaky_call, attempts=5, base_delay=0.2))

## 3) Как обернуть LLM‑вызов

Здесь тот же принцип: если ошибка временная — повторяем.

In [ ]:
# Пример-скелет (подключи свой llm)
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-5.2-chat", temperature=0.2)

# def llm_call():
#     return llm.invoke("Что такое RAG?").content

# print(retry_backoff(llm_call, attempts=4, base_delay=1.0))